In [28]:
from skimage.io import imread
from skimage import img_as_float
import pylab
from sklearn.cluster import KMeans
import pandas as pd
from math import log10

In [29]:
# Загрузите картинку parrots.jpg. Преобразуйте изображение, 
# приведя все значения в интервал от 0 до 1. Для этого можно 
# воспользоваться функцией img_as_float из модуля skimage.
# Обратите внимание на этот шаг, так как при работе с исходным 
# изображением вы получите некорректный результат.

image = imread("parrots.jpg")
# pylab.imshow(image)

In [30]:
img_float = img_as_float(image)
# img_float

In [31]:
# Создайте матрицу объекты-признаки: характеризуйте каждый пиксель
# тремя координатами - значениями интенсивности в пространстве RGB.

img_matrix = {}
i = 0
for x in range(len(img_float)):
    for y in range(len(img_float)):
        img_matrix[i] = img_float[x][y]
        i += 1
# img_matrix

In [32]:
X_train = pd.DataFrame(img_matrix).transpose()
X_train.columns = ["R", "G", "B"]
reall = X_train.copy()

In [37]:
# Запустите алгоритм K-Means с параметрами init='k-means++' и random_state=241.
# После выделения кластеров все пиксели, отнесенные в один кластер, попробуйте заполнить двумя способами:
# медианным и средним цветом по кластеру.

def PSNR(X, X_train_real):
    m = 0
    for col in X_train_real.columns:
        m += ((X[col] - X_train_real[col])**2).sum()
    m /= len(X_train_real) * 3
    
    return 10 * log10(1 / m)

In [39]:
for c in range(21):
    X_train = reall.copy()
    max_c = c + 1
    
    cls = KMeans(init = 'k-means++', random_state = 241, n_clusters = max_c)
    kmeans = cls.fit(X_train)
    
    X_train['cluster'] = kmeans.labels_
    X_train.set_index('cluster', inplace=True)
    X_train2 = X_train.copy()
    X_train_real = X_train.copy()
    
    for cluster in range(max_c):
        for col in X_train.columns:
            median=X_train.loc[cluster,col].median()#считаем медиану интенсивности по кластеру
            X_train.loc[cluster,col]=median

            mean=X_train2.loc[cluster,col].mean()#считаем среднее интенсивности по кластеру
            X_train2.loc[cluster,col]=mean
    
    psnrMedian=PSNR(X_train,X_train_real)
    psnrMean=PSNR(X_train2,X_train_real)

    print("Clusters count %d, psnrMedian=%f, psnrMean=%f" % (max_c,psnrMedian,psnrMean))
    if (psnrMedian>=20) | (psnrMean>=20): break

Clusters count 1, psnrMedian=9.517311, psnrMean=10.004023
Clusters count 2, psnrMedian=11.907260, psnrMean=12.285052
Clusters count 3, psnrMedian=13.173261, psnrMean=13.639986
Clusters count 4, psnrMedian=14.341385, psnrMean=14.848330
Clusters count 5, psnrMedian=15.513345, psnrMean=16.087328
Clusters count 6, psnrMedian=16.494179, psnrMean=17.186362
Clusters count 7, psnrMedian=17.913883, psnrMean=18.213271
Clusters count 8, psnrMedian=18.803852, psnrMean=19.037773
Clusters count 9, psnrMedian=19.387685, psnrMean=19.584089
Clusters count 10, psnrMedian=19.899284, psnrMean=20.113454
